In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/polynomial_train.csv",index_col = 0)
df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/polynomial_test_data.csv")

train_data = df_train.iloc[:45000,:-1].to_numpy()
X_test = df_train.iloc[45000:50000,:-1].to_numpy()
label_train = df_train["label"].values[:45000]
label_test = df_train["label"].values[45000:50000]
deg = 5
print(label_test)

#standardization

X = (train_data - np.mean(train_data,axis= 0 ))/np.std(train_data,axis= 0)
X_test = (X_test - np.mean(train_data,axis= 0 ))/np.std(train_data,axis= 0)
y = (label_train - np.mean(label_train,axis = 0))/np.std(label_train,axis = 0)
y_test = (label_test - np.mean(label_train,axis = 0))/np.std(label_train,axis = 0)

print((X[:,0]**2*X[:,1]).shape)

def poly(X,degree):
    
    x = np.ones(X.shape[0])
    for i in range(degree+1):
      for j in range(degree+1):
        for k in range(degree+1):
            if(i+j+k<=degree):
                x = np.vstack([x,X[:,0]**i*X[:,1]**j*X[:,2]**k])
    x = x[1:,:].T
    return x

x = poly(X,deg)
x_test = poly(X_test,deg)

print(x.shape)

[-6895.18223704 -2416.42109958 36683.04220994 ...  -285.70796526
  8161.59530383  -255.16713134]
(45000,)
(45000, 56)


In [3]:

def Plot(x,y,x_test,y_test):
    plt.scatter(x,y,c ='k')
    plt.scatter(x_test,y_test,c='r')
    plt.xlabel("X -->")
    plt.ylabel("Y -->")
    plt.grid(True)
    plt.show()


In [4]:
def R2(x_test, m , y_test):
    y_pred = x_test @ m
    score = 1 - np.sum((y_pred-y_test)**2)/np.sum((y_test-np.mean(y_test))**2)
    return score 

In [18]:

def ML(no_of_iterations,learning_rate,x, y , x_test, y_test):
    row,col = x.shape
    first,last = 0,0
    m = np.zeros(col)
    b = row 
    m = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Polynomial_m.npy")

    for i in range(no_of_iterations+1):
        y_pred = np.dot(x,m)
        loss = 0.5/b*np.sum((y_pred-y)**2)
        
        m = m - learning_rate/b*np.dot(x.T,y_pred- y)
        loss = 0.5/b*np.sum((y_pred-y)**2)
      
        if (i % 1000) == 0 :
            print("After ",i+1, " iterations: ")
            first = last
            last = loss
            score = R2(x_test, m , y_test)
            print("loss = ",loss,", learning rate = ",learning_rate 
                  ," Score = ",score*100,"%")
            Plot(x[:45000,1],y,x[:45000,1],y_pred)
            np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Polynomial_m.npy",m)
            print(m)
            
    Plot(x[:,1],y,x[:,1],y_pred)
    plt.show()
    return m 

param = ML(no_of_iterations=1000000,learning_rate=1e-5,x= x,y= y
           , x_test= x_test,y_test=y_test)



Output hidden; open in https://colab.research.google.com to view.

In [19]:
test_data = df_test.iloc[:,1:].to_numpy()

X = (test_data - np.mean(train_data,axis=0))/np.std(train_data,axis=0)

x_test = poly(X,deg)
param = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Polynomial_m.npy")

ids = df_test.iloc[:,0].values
y_test = np.dot(x_test,param)
print(y_test)
y_res = y_test * np.std(label_train,axis = 0) + np.mean(label_train,axis = 0)
res = pd.DataFrame([ids,y_res],index = ['Ids','Output']).T

print(res)
#res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Polynomial_Regression_result.csv")


[-0.00666863  0.09004794 -0.00445769 ... -0.00647807 -0.00653709
 -0.05034503]
           Ids        Output
0      78886.0 -5.440630e+02
1      35987.0  2.430256e+05
2      63576.0  5.023952e+03
3      86537.0 -2.059091e+06
4      45235.0 -3.644414e+04
...        ...           ...
19995  42763.0 -1.309074e+04
19996  52748.0 -6.480889e+02
19997  42159.0 -6.415892e+01
19998  22095.0 -2.127978e+02
19999  80712.0 -1.105381e+05

[20000 rows x 2 columns]
